<a href="https://colab.research.google.com/github/arlianisarumy/Clickbait-Detection-Using-FastText/blob/main/modelling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fasttext
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199776 sha256=9718610f24781fb0ab55f1e1ddfc52f5b8caa2fd8b8a5f0f4b53901d716b78ef
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [2]:
import nltk, warnings; warnings.simplefilter('ignore')
import random
import numpy as np
import fasttext
import csv
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import plotly.express as px
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/skripsi_arum/data/data_modelling1.csv')

In [5]:
df.head()

,title,label,label_score,title_cleaned
0,"Masuk Radar Pilwalkot Medan, Menantu Jokowi Be...",non-clickbait,0,masuk radar pilwalkot medan menantu jokowi tem...
1,Malaysia Sudutkan RI: Isu Kabut Asap hingga In...,non-clickbait,0,malaysia sudut ri isu kabut asap invasi babi
2,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,clickbait,1,viral driver ojol bekas pesan makan pakai sepeda
3,"Kemensos Salurkan Rp 7,3 M bagi Korban Kerusuh...",non-clickbait,0,kemensos salur rp korban rusuh sosial papua
4,"Terkait Mayat Bayi Mengenaskan di Tangerang, S...",non-clickbait,0,kait mayat bayi enas tangerang pria tangkap po...


In [6]:
data = df[['title_cleaned','label_score']]
data.head()

,title_cleaned,label_score
0,masuk radar pilwalkot medan menantu jokowi tem...,0
1,malaysia sudut ri isu kabut asap invasi babi,0
2,viral driver ojol bekas pesan makan pakai sepeda,1
3,kemensos salur rp korban rusuh sosial papua,0
4,kait mayat bayi enas tangerang pria tangkap po...,0


In [7]:
X = data.title_cleaned.values.tolist()
y = data.label_score.values.tolist()

In [8]:
data_utama, test_X, label_utama, test_y = train_test_split(X, y, test_size=0.20, stratify = y, random_state=42)

In [9]:
label = ['__label__'+ str(y) for y in label_utama]
label_test = ['__label__'+ str(y) for y in test_y]

In [10]:
data2=pd.DataFrame({'label_score': label,'text': data_utama})

In [11]:
label_title = data2['label_score'] + " " + data2['text']

In [12]:
label_title=label_title.tolist()

In [13]:
label_title

['__label__0 bursa transfer liga tutup pss sleman datang main',
 '__label__0 diagnosis kembung bocah tinggal akibat kanker perut',
 '__label__0 indonesia bambu olah energi listrik rang desa',
 '__label__0 caleg perempuan kian kuat milu',
 '__label__1 singgung makna lagu kasih sayang orangtua mawang',
 '__label__1 brakk truk ngebut lindas kendara motor jalan gatsu medan',
 '__label__0 perintah terima ganti rugi usaha bakar lahan',
 '__label__1 demo surabaya cenderung kondusif duga alas',
 '__label__0 icw kritik pidato jokowi revisi uu kpk kuat',
 '__label__1 huawei mate pro spesifikasi gambar',
 '__label__0 nunung jefri nichol tangkap atta halilintar keren narkoba',
 '__label__1 massa gejayan panggil kumpul singgung rezim pinokio',
 '__label__1 main film horor prisia nasution syuting jam',
 '__label__1 tiga final gala catur emas penuh kejut',
 '__label__0 toyota indonesia lulus siswa pakai',
 '__label__1 foto bareng wanita risa saraswati minder',
 '__label__1 google tantang tumbuh start

In [14]:
param_grid = {
    'lr': [0.001, 0.01, 0.1, 1],
    'wordNgrams': [2, 3, 4, 5],
    'epoch': [5, 10, 15, 20]
}
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
best_score = 0
best_params = {}

In [15]:
fold_accuracies = []
results = []
for lr in param_grid['lr']:
    for wordNgrams in param_grid['wordNgrams']:
        for epoch in param_grid['epoch']:
          avg_accuracy = 0
          fold_accuracy_list = []
          for train_idx, val_idx in skf.split(np.zeros(len(label_title)), np.zeros(len(label_title))):
              train_data = [label_title[i] for i in train_idx]
              val_data = [label_title[i] for i in val_idx]

              # Simpan data ke file temporary (FastText membutuhkan file)
              with open('train.txt', 'w') as f:
                  f.write('\n'.join(train_data))
              with open('val.txt', 'w') as f:
                  f.write('\n'.join(val_data))

              # Training dengan FastText
              model = fasttext.train_supervised('train.txt', lr=lr, wordNgrams=wordNgrams, epoch=epoch)

              # Evaluasi model
              result = model.test('val.txt')
              accuracy = result[1]
              avg_accuracy += accuracy
              fold_accuracy_list.append(accuracy)

          avg_accuracy /= k_folds

          results.append({'lr': lr, 'wordNgrams': wordNgrams, 'epoch': epoch,'Accuracy': avg_accuracy})
          print("lr = ", lr, "wordNgrams = ", wordNgrams, "epoch = ", epoch, "akurasi = ", avg_accuracy)
          print("fold =", fold_accuracy_list)

          if avg_accuracy > best_score:
              best_score = avg_accuracy
              best_params = {'lr': lr, 'wordNgrams': wordNgrams, 'epoch': epoch}

print("Best Parameters:", best_params)
print("Best Average Accuracy:", best_score)

# Create DataFrame from results list
results_df = pd.DataFrame(results)

lr =  0.001 wordNgrams =  2 epoch =  5 akurasi =  0.5807499999999999
fold = [0.58875, 0.5908333333333333, 0.5675, 0.58125, 0.5754166666666667]
lr =  0.001 wordNgrams =  2 epoch =  10 akurasi =  0.5807499999999999
fold = [0.58875, 0.5908333333333333, 0.5675, 0.58125, 0.5754166666666667]
lr =  0.001 wordNgrams =  2 epoch =  15 akurasi =  0.5806666666666667
fold = [0.58875, 0.5908333333333333, 0.5675, 0.5808333333333333, 0.5754166666666667]
lr =  0.001 wordNgrams =  2 epoch =  20 akurasi =  0.5806666666666667
fold = [0.58875, 0.5908333333333333, 0.5675, 0.5808333333333333, 0.5754166666666667]
lr =  0.001 wordNgrams =  3 epoch =  5 akurasi =  0.5805833333333335
fold = [0.5883333333333334, 0.5908333333333333, 0.5675, 0.58125, 0.575]
lr =  0.001 wordNgrams =  3 epoch =  10 akurasi =  0.5808333333333333
fold = [0.58875, 0.5908333333333333, 0.5675, 0.58125, 0.5758333333333333]
lr =  0.001 wordNgrams =  3 epoch =  15 akurasi =  0.5807499999999999
fold = [0.58875, 0.5908333333333333, 0.5675, 0.5

In [16]:
results_df.head()

,lr,wordNgrams,epoch,Accuracy
0,0.001,2,5,0.580750
1,0.001,2,10,0.580750
2,0.001,2,15,0.580667
3,0.001,2,20,0.580667
4,0.001,3,5,0.580583


In [17]:
results_df['lr_log'] = np.log10(results_df['lr'])

In [18]:
results_df

,lr,wordNgrams,epoch,Accuracy,lr_log
0,0.001,2,5,0.580750,-3.0
1,0.001,2,10,0.580750,-3.0
2,0.001,2,15,0.580667,-3.0
3,0.001,2,20,0.580667,-3.0
4,0.001,3,5,0.580583,-3.0
...,...,...,...,...,...
59,1.000,4,20,0.722583,0.0
60,1.000,5,5,0.723167,0.0
61,1.000,5,10,0.724083,0.0
62,1.000,5,15,0.726083,0.0


In [19]:
# Membuat parallel coordinates plot
fig = px.parallel_coordinates(results_df, color='lr_log', labels={'Accuracy': 'Akurasi', 'lr_log': 'lr'},
                              dimensions=['wordNgrams', 'lr_log', 'epoch', 'Accuracy'],
                              color_continuous_scale=px.colors.sequential.Magma)

fig.show()

In [20]:
y_predict = [model.predict(text)[0][0] for text in test_X]

In [21]:
confusion_matrix(label_test, y_predict)

array([[1427,  315],
       [ 525,  733]])

In [22]:
print(classification_report(label_test, y_predict, target_names=['non-clickbait','clickbait']))

               precision    recall  f1-score   support

non-clickbait       0.73      0.82      0.77      1742
    clickbait       0.70      0.58      0.64      1258

     accuracy                           0.72      3000
    macro avg       0.72      0.70      0.70      3000
 weighted avg       0.72      0.72      0.72      3000

